In [2]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [40]:
import findspark

In [4]:
import time
import numpy as np

In [5]:
s

In [6]:
df = sc.read.json("nyt2.json")

In [7]:
df.show()

+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|        price|   published_date|           publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|              Bantam| {1}|           {0}|           ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{1212883200000}}|       Little, Br

In [9]:
df.dtypes

[('_id', 'struct<$oid:string>'),
 ('amazon_product_url', 'string'),
 ('author', 'string'),
 ('bestsellers_date', 'struct<$date:struct<$numberLong:string>>'),
 ('description', 'string'),
 ('price', 'struct<$numberDouble:string,$numberInt:string>'),
 ('published_date', 'struct<$date:struct<$numberLong:string>>'),
 ('publisher', 'string'),
 ('rank', 'struct<$numberInt:string>'),
 ('rank_last_week', 'struct<$numberInt:string>'),
 ('title', 'string'),
 ('weeks_on_list', 'struct<$numberInt:string>')]

In [12]:
df.head()

Row(_id=Row($oid='5b4aa4ead3089013507db18b'), amazon_product_url='http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20', author='Dean R Koontz', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.', price=Row($numberDouble=None, $numberInt='27'), published_date=Row($date=Row($numberLong='1212883200000')), publisher='Bantam', rank=Row($numberInt='1'), rank_last_week=Row($numberInt='0'), title='ODD HOURS', weeks_on_list=Row($numberInt='1'))

In [11]:
df.describe().show()

+-------+--------------------+---------------+--------------------+---------+------------------+
|summary|  amazon_product_url|         author|         description|publisher|             title|
+-------+--------------------+---------------+--------------------+---------+------------------+
|  count|               10195|          10195|               10195|    10195|             10195|
|   mean|                null|           null|                null|     null|1877.7142857142858|
| stddev|                null|           null|                null|     null| 370.9760613506458|
|    min|http://www.amazon...|        AJ Finn|                    |      ACE|  10TH ANNIVERSARY|
|    max|https://www.amazo...|various authors|’Tis for the Rebe...|allantine|               ZOO|
+-------+--------------------+---------------+--------------------+---------+------------------+



In [13]:
df.count()

10195

In [19]:
df.distinct().show(5)

+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+------------------+-------------+
|                 _id|  amazon_product_url|         author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|             title|weeks_on_list|
+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|   Daniel Silva|{{1217030400000}}|Gabriel Allon, an...|{26.95, null}|{{1218326400000}}|       Putnam| {1}|           {0}|THE SECRET SERVANT|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|     Jane Green|{{1218240000000}}|A woman’s life ch...|    {null, 0}|{{1219536000000}}|       Viking|{18}|           {0}|   THE BEACH HOUSE|          {0}|
|{5b4aa4ea

In [16]:
df.take(2)

[Row(_id=Row($oid='5b4aa4ead3089013507db18b'), amazon_product_url='http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20', author='Dean R Koontz', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.', price=Row($numberDouble=None, $numberInt='27'), published_date=Row($date=Row($numberLong='1212883200000')), publisher='Bantam', rank=Row($numberInt='1'), rank_last_week=Row($numberInt='0'), title='ODD HOURS', weeks_on_list=Row($numberInt='1')),
 Row(_id=Row($oid='5b4aa4ead3089013507db18c'), amazon_product_url='http://www.amazon.com/The-Host-Novel-Stephenie-Meyer/dp/0316218502?tag=NYTBS-20', author='Stephenie Meyer', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Aliens have taken control of the minds and bodies of most humans, but one woman won’t surrender.', price=Row($numberDouble='25.99', $numberInt=None), published_date=

In [17]:
df_no_dup = df.dropDuplicates()

In [18]:
df_no_dup.count()

10195

In [21]:
df.select("author","title").show(5)

+-----------------+--------------------+
|           author|               title|
+-----------------+--------------------+
|    Dean R Koontz|           ODD HOURS|
|  Stephenie Meyer|            THE HOST|
|     Emily Giffin|LOVE THE ONE YOU'...|
|Patricia Cornwell|           THE FRONT|
|  Chuck Palahniuk|               SNUFF|
+-----------------+--------------------+
only showing top 5 rows



In [23]:
df.select("title", when (df.title == "THE HOST", 1).otherwise(0)).show(4)

+--------------------+----------------------------------------------+
|               title|CASE WHEN (title = THE HOST) THEN 1 ELSE 0 END|
+--------------------+----------------------------------------------+
|           ODD HOURS|                                             0|
|            THE HOST|                                             1|
|LOVE THE ONE YOU'...|                                             0|
|           THE FRONT|                                             0|
+--------------------+----------------------------------------------+
only showing top 4 rows



In [24]:
df[df.title.isin("ODD HOURS")].show(5)

+--------------------+--------------------+-------------+-----------------+--------------------+----------+-----------------+---------+----+--------------+---------+-------------+
|                 _id|  amazon_product_url|       author| bestsellers_date|         description|     price|   published_date|publisher|rank|rank_last_week|    title|weeks_on_list|
+--------------------+--------------------+-------------+-----------------+--------------------+----------+-----------------+---------+----+--------------+---------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|{null, 27}|{{1212883200000}}|   Bantam| {1}|           {0}|ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|Dean R Koontz|{{1212192000000}}|Odd Thomas, who c...|{null, 27}|{{1213488000000}}|   Bantam| {3}|           {1}|ODD HOURS|          {2}|
|{5b4aa4ead3089013...|http://www.amazon...|Dean R Koontz|{{1212796800000}}|Odd Thomas, who c...|{nul

In [25]:
df.select("author", "title",df.title.startswith("THE")).show(5)

+-----------------+--------------------+----------------------+
|           author|               title|startswith(title, THE)|
+-----------------+--------------------+----------------------+
|    Dean R Koontz|           ODD HOURS|                 false|
|  Stephenie Meyer|            THE HOST|                  true|
|     Emily Giffin|LOVE THE ONE YOU'...|                 false|
|Patricia Cornwell|           THE FRONT|                  true|
|  Chuck Palahniuk|               SNUFF|                 false|
+-----------------+--------------------+----------------------+
only showing top 5 rows



In [27]:
df.select("author", "title",df.title.like("% ONE %")).show(5)


+-----------------+--------------------+------------------+
|           author|               title|title LIKE % ONE %|
+-----------------+--------------------+------------------+
|    Dean R Koontz|           ODD HOURS|             false|
|  Stephenie Meyer|            THE HOST|             false|
|     Emily Giffin|LOVE THE ONE YOU'...|              true|
|Patricia Cornwell|           THE FRONT|             false|
|  Chuck Palahniuk|               SNUFF|             false|
+-----------------+--------------------+------------------+
only showing top 5 rows



In [28]:
df = df.withColumnRenamed("title", "TITLE")

In [29]:
df_rem = df.drop("published_date")

In [30]:
df_rem.show(4)

+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+-------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|           author| bestsellers_date|         description|        price|    publisher|rank|rank_last_week|               TITLE|weeks_on_list|
+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+-------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|       Bantam| {1}|           {0}|           ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|Little, Brown| {2}|           {1}|            THE HOST|          {3}|
|{5b4aa4ead3089013...|http://www.amazon...|     Emily Giffin|{{1211587200000}}|A

In [33]:
df.groupby("author").count().show()

+--------------------+-----+
|              author|count|
+--------------------+-----+
|          James Frey|    2|
|    Elin Hilderbrand|   58|
|   Sharon Kay Penman|    2|
|         Lisa Genova|    7|
|        Will Allison|    1|
|   Patricia Cornwell|   64|
|       Laurie R King|    6|
|          Tea Obreht|    6|
|        Sarah Dunant|    1|
|        Tim Johnston|    1|
|          Sara Gruen|   13|
|Tom Clancy with P...|    9|
|         Andre Dubus|    1|
|        Terry Brooks|   19|
|        Daniel Silva|   69|
|      Jackie Collins|   14|
|          Pat Conroy|   11|
|          Ann B Ross|    5|
|      Michael Savage|    3|
|David Weber and E...|    1|
+--------------------+-----+
only showing top 20 rows



In [36]:
df.filter(df["title"]== "ODD HOURS").show(4)

+--------------------+--------------------+-------------+-----------------+--------------------+----------+-----------------+---------+----+--------------+---------+-------------+
|                 _id|  amazon_product_url|       author| bestsellers_date|         description|     price|   published_date|publisher|rank|rank_last_week|    TITLE|weeks_on_list|
+--------------------+--------------------+-------------+-----------------+--------------------+----------+-----------------+---------+----+--------------+---------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|{null, 27}|{{1212883200000}}|   Bantam| {1}|           {0}|ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|Dean R Koontz|{{1212192000000}}|Odd Thomas, who c...|{null, 27}|{{1213488000000}}|   Bantam| {3}|           {1}|ODD HOURS|          {2}|
|{5b4aa4ead3089013...|http://www.amazon...|Dean R Koontz|{{1212796800000}}|Odd Thomas, who c...|{nul

In [37]:
df.registerTempTable("df3")

C:\Users\HP\anaconda3\lib\site-packages\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [39]:
sc.sql("select * from df3 where title = 'ODD HOURS'").show(3)

+--------------------+--------------------+-------------+-----------------+--------------------+----------+-----------------+---------+----+--------------+---------+-------------+
|                 _id|  amazon_product_url|       author| bestsellers_date|         description|     price|   published_date|publisher|rank|rank_last_week|    TITLE|weeks_on_list|
+--------------------+--------------------+-------------+-----------------+--------------------+----------+-----------------+---------+----+--------------+---------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|{null, 27}|{{1212883200000}}|   Bantam| {1}|           {0}|ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|Dean R Koontz|{{1212192000000}}|Odd Thomas, who c...|{null, 27}|{{1213488000000}}|   Bantam| {3}|           {1}|ODD HOURS|          {2}|
|{5b4aa4ead3089013...|http://www.amazon...|Dean R Koontz|{{1212796800000}}|Odd Thomas, who c...|{nul